In [1]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer, make_column_selector

from sklearn.model_selection import cross_validate

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

import numpy as np
import os
import pandas as pd

# Data


In [11]:
data_path = "../data"
DATA = []
for file in os.listdir(data_path):
    DATA.append(pd.read_csv(os.path.join(data_path,file)))

In [12]:
for data in DATA:
    print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        5000 non-null   int64 
 1   job        5000 non-null   object
 2   marital    5000 non-null   object
 3   education  5000 non-null   object
 4   default    5000 non-null   object
 5   balance    5000 non-null   int64 
 6   housing    5000 non-null   object
 7   loan       5000 non-null   object
 8   contact    5000 non-null   object
 9   day        5000 non-null   int64 
 10  month      5000 non-null   object
 11  duration   5000 non-null   int64 
 12  campaign   5000 non-null   int64 
 13  pdays      5000 non-null   int64 
 14  previous   5000 non-null   int64 
 15  poutcome   5000 non-null   object
 16  y          5000 non-null   object
dtypes: int64(7), object(10)
memory usage: 664.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 

# Pipeline

In [4]:
num_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer()),
    ('scale', MinMaxScaler())
])

cat_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy="most_frequent")),
    ('one-hot', OneHotEncoder(handle_unknown='ignore'))
])


col_trans = ColumnTransformer([
    ('num_pipeline', num_pipeline, make_column_selector(dtype_include = np.number)),
    ('cat_pipeline', cat_pipeline, make_column_selector(dtype_include = np.object_))
])


In [5]:
classifiers = [
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    XGBClassifier(),
]

pipelines = []

In [6]:
for classifier in classifiers:
    pipelines.append((type(classifier), Pipeline([("transformer", col_trans), ("model", classifier)])))

In [ ]:
pipelines

In [ ]:
from sklearn.calibration import LabelEncoder


scores = []

for pipe in pipelines:
    pipe_score = []
    for data in DATA:
        score = cross_validate(pipe[1], data.iloc[:, :-1], LabelEncoder().fit_transform(data.iloc[:,-1]), cv = 5, scoring="accuracy")
        pipe_score.append(score["test_score"].mean())

    scores.append((pipe[0], pipe_score))

In [10]:
os.listdir("../data")

['banking_final.csv',
 'flights_final.csv',
 'history_dataset_DecisionTree.csv',
 'history_dataset_RandomForest.csv',
 'history_dataset_XGBoost.csv',
 'mushrooms_final.csv',
 'weather_final.csv']

In [ ]:
scores

# Random Searching - searching for new defaults

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

param_distributions = [
    {
        "model__max_depth": randint(1,30),
        "model__min_samples_split":randint(2,60),
        "model__min_impurity_decrease":uniform(0,1),
        "model__min_samples_leaf":randint(1,60)
    },
    {
        "model__n_estimators":randint(1,2000),
        "model__min_samples_leaf":randint(1,5000),
        "model__max_samples":uniform(0.1,1),
        "model__max_features":uniform(0,1)
    },
    {
        "model__max_depth": randint(1,15),
        "model__min_child_weight": randint(1,128),
        "model__eta": uniform(2**(-10),1),
        "model__alpha": uniform(2**(-10),2**(10))
    }
]
best_params = [[],[],[],[]]
pipe_best_models = []
pipe_best_scores = []
history = [[],[],[]]
for i,pipe in enumerate(pipelines):    
    for j,data in enumerate(DATA):
        rs = RandomizedSearchCV(pipe[1], 
                                param_distributions= param_distributions[i],
                                verbose=True,
                                random_state=42,
                                cv=5,
                                n_iter=200
                                )
        rs.fit(data.iloc[:, :-1],LabelEncoder().fit_transform(data.iloc[:,-1]))
        pipe_best_scores.append(rs.best_score_)
        pipe_best_models.append(rs.best_estimator_)
        best_params[j].append(rs.best_params_)
        history[i].append(rs.cv_results_)    


In [ ]:
history_datasets = []
for h in history:
    df = pd.concat([pd.DataFrame(h[i]) for i in range(len(h))], keys=range(len(h)), names=['dataset'])
    df = df.reset_index()
    df.drop(columns='level_1', inplace=True)
    history_datasets.append(df)
print(f"{pipelines[0][0]} shape: {history_datasets[0].shape}")
print(f"{pipelines[1][0]} shape: {history_datasets[1].shape}")
print(f"{pipelines[2][0]} shape: {history_datasets[2].shape}")

In [55]:
model_names = ['DecisionTree','RandomForest','XGBoost']

# Saving history to csv files

In [58]:
for i, df in enumerate(history_datasets):
    df.to_csv(f'../history/history_dataset_{model_names[i]}.csv', index=False)

# Reading history from csv

In [18]:
history_DecisionTree = pd.read_csv('../history/history_dataset_DecisionTree.csv')
history_RandomForest = pd.read_csv('../history/history_dataset_RandomForest.csv')
history_XGBoost = pd.read_csv('../history/history_dataset_XGBoost.csv')

In [15]:
history_DecisionTree.head()

,dataset,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__criterion,param_model__max_depth,param_model__min_samples_leaf,param_model__min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0,0.034176,0.006215,0.004794,0.002535,gini,20,29,16,"{'model__criterion': 'gini', 'model__max_depth...",0.902,0.894,0.891,0.884,0.886,0.8914,0.006375,121
1,0,0.028392,0.001495,0.004094,0.002321,gini,8,21,40,"{'model__criterion': 'gini', 'model__max_depth...",0.904,0.889,0.885,0.887,0.889,0.8908,0.006765,134
2,0,0.031333,0.001139,0.005442,0.002487,entropy,19,23,12,"{'model__criterion': 'entropy', 'model__max_de...",0.889,0.898,0.895,0.890,0.874,0.8892,0.008280,151
3,0,0.026831,0.003970,0.006345,0.003212,gini,24,53,37,"{'model__criterion': 'gini', 'model__max_depth...",0.901,0.903,0.891,0.896,0.891,0.8964,0.004964,24
4,0,0.034022,0.001552,0.007549,0.001566,entropy,24,3,23,"{'model__criterion': 'entropy', 'model__max_de...",0.887,0.890,0.872,0.881,0.870,0.8800,0.007925,200


In [21]:
history_DecisionTree.shape

(800, 18)

In [ ]:
history_RandomForest.head()

In [ ]:
history_XGBoost.head()

# New defaults below

In [26]:
def get_best_params_overall(df):
    df['params_str'] = df['params'].apply(lambda x: str(x))
    grouped_mean = df.groupby(['params_str'])['mean_test_score'].mean().reset_index()
    grouped_mean.sort_values(by='mean_test_score', ascending=False, inplace=True)
    return grouped_mean.iloc[0, 0], grouped_mean.iloc[0, 1]

In [31]:
best_params_DecisionTree, best_params_DecisionTree_score = get_best_params_overall(history_DecisionTree)
print(f"Best params for DecisionTree: {best_params_DecisionTree}") 
print(f"with score: {best_params_DecisionTree_score}")

Best params for DecisionTree: {'model__criterion': 'entropy', 'model__max_depth': 28, 'model__min_samples_leaf': 5, 'model__min_samples_split': 48}
with score: 0.8935500000000001


In [32]:
best_params_RandomForest, best_params_RandomForest_score = get_best_params_overall(history_RandomForest)
print(f"Best params for RandomForest: {best_params_RandomForest}") 
print(f"with score: {best_params_RandomForest_score}")

Best params for RandomForest: {'model__max_features': 0.9191765518355596, 'model__min_samples_leaf': 10, 'model__min_samples_split': 57, 'model__n_estimators': 798}
with score: 0.90745


In [33]:
best_params_XGBoost, best_params_XGBoost_score = get_best_params_overall(history_XGBoost)
print(f"Best params for XGBoost: {best_params_XGBoost}")
print(f"With score: {best_params_XGBoost_score}")

Best params for XGBoost: {'model__alpha': 14.73990891937001, 'model__eta': 0.11607264050691624, 'model__max_depth': 5, 'model__min_child_weight': 47}
With score: 0.88395


# Tunability

##### Now let's compute tunability of each of the ML algorithms. We'll start with looking for the optimal configuration of the hyperparameters for each of the datasets

In [84]:
def get_best_params_per_dataset(df):
    df['params_str'] = df['params'].apply(lambda x: str(x))
    best_params_per_dataset = df.sort_values(['dataset', 'rank_test_score'], ascending=[True, True]).groupby('dataset').first().reset_index()
    best_params_per_dataset.rename(columns={'params_str': 'best_params', 'mean_test_score': 'best_score'}, inplace=True)
    best_params_per_dataset = best_params_per_dataset[['dataset', 'best_params', 'best_score']]
    default_params, _ = get_best_params_overall(df)
    score_for_default_params = df[df['params_str'] == default_params][['dataset', 'mean_test_score']].rename(columns={'mean_test_score': 'default_score'})
    best_params_per_dataset = best_params_per_dataset.merge(score_for_default_params, on='dataset', how='left')
    best_params_per_dataset['abs_tunability'] = best_params_per_dataset['best_score'] - best_params_per_dataset['default_score']
    best_params_per_dataset['rel_tunability (%)'] = best_params_per_dataset['abs_tunability'] / best_params_per_dataset['default_score'] * 100 
    return best_params_per_dataset

In [85]:
best_params_per_dataset_DecisionTree = get_best_params_per_dataset(history_DecisionTree)
best_params_per_dataset_DecisionTree

,dataset,best_params,best_score,default_score,abs_tunability,rel_tunability (%)
0,0,"{'model__criterion': 'gini', 'model__max_depth...",0.8992,0.8882,0.0110,1.238460
1,1,"{'model__criterion': 'entropy', 'model__max_de...",0.9240,0.9218,0.0022,0.238663
2,2,"{'model__criterion': 'entropy', 'model__max_de...",0.9742,0.9532,0.0210,2.203105
3,3,"{'model__criterion': 'gini', 'model__max_depth...",0.8276,0.8110,0.0166,2.046856


In [86]:
best_params_per_dataset_RandomForest = get_best_params_per_dataset(history_RandomForest)
best_params_per_dataset_RandomForest

,dataset,best_params,best_score,default_score,abs_tunability,rel_tunability (%)
0,0,"{'model__max_features': 0.9191765518355596, 'm...",0.9004,0.9004,0.0,0.0
1,1,"{'model__max_features': 0.9191765518355596, 'm...",0.9330,0.9330,0.0,0.0
2,2,"{'model__max_features': 0.9191765518355596, 'm...",0.9600,0.9600,0.0,0.0
3,3,"{'model__max_features': 0.9191765518355596, 'm...",0.8364,0.8364,0.0,0.0


In [87]:
best_params_per_dataset_XGBoost = get_best_params_per_dataset(history_XGBoost)
best_params_per_dataset_XGBoost

,dataset,best_params,best_score,default_score,abs_tunability,rel_tunability (%)
0,0,"{'model__alpha': 5.655624497068857, 'model__et...",0.8962,0.8906,0.0056,0.628790
1,1,"{'model__alpha': 5.655624497068857, 'model__et...",0.9276,0.9172,0.0104,1.133886
2,2,"{'model__alpha': 14.73990891937001, 'model__et...",0.8902,0.8902,0.0000,0.000000
3,3,"{'model__alpha': 14.73990891937001, 'model__et...",0.8378,0.8378,0.0000,0.000000
